In [6]:
import numpy as np
import pandas as pd
# from GRNN import GRNN, calculate_error_cost
from gtm_steps import MultiLabelGTM
from scipy import stats
from scipy.optimize import differential_evolution
from sklearn.base import clone
from sklearn.decomposition import PCA
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, Normalizer, PowerTransformer, RobustScaler, StandardScaler, PolynomialFeatures
from sklearn.metrics import max_error, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, median_absolute_error, r2_score
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.linear_model import Ridge

In [7]:
# Choose task
task_name = 'Task1' # Task1 Task2
additional_notes = '_best_results'

In [8]:
class ModelStrategy(object):
    def __init__(self, regressor) -> None:
        self._strategy = regressor
        self._regressors = []

    def get_strategy(self):
        return self._strategy

    def get_regressors(self):
        return self._regressors

    def set_strategy(self, regressor) -> None:
        self._strategy = regressor

    def fit(self, X_train, Y_train) -> None:
        for y_train in Y_train.T:
            regressor = clone(self._strategy, safe=True)
            # Fit and save all regressors
            regressor.fit(X_train, y_train)
            self._regressors.append(regressor)

    def fit_transform(self, X_train, Y_train):
        y_train_pred = []
        for y_train in Y_train.T:
            regressor = clone(self._strategy, safe=True)
            # Fit and save all regressors
            regressor.fit(X_train, y_train)
            self._regressors.append(regressor)
            # Predict and save prediction for future use
            y_train_pred.append(regressor.predict(X_train))

        # Transfrom data to numpy array and return
        return np.stack(y_train_pred, axis=1)

    def transform(self, X):
        y_pred = []
        for regressor in self._regressors:
            # For each saved regressor predict an output value
            y_pred.append(regressor.predict(X))
        return np.stack(y_pred, axis=1)

class AdapterSGTM(object):
    def __init__(self, number_of_steps = 1000, verbose=0):
        self._gtm = MultiLabelGTM(number_of_steps = number_of_steps, center_of_mass = True, verbose=verbose)

    def fit(self, X_train) -> None:
        self._gtm.fit(X_train, X_train)

    def fit_transform(self, X_train):
        self._gtm.fit(X_train, X_train)
        return self._gtm.predict(X_train)

    def transform(self, X):
        return self._gtm.predict(X)


def relative_root_mean_squared_error(y, y_pred):
    n = len(y)
    num = np.sum(np.square(y - y_pred)) / n
    den = np.sum(np.square(y_pred))
    squared_error = num / den
    rrmse_loss = np.sqrt(squared_error)
    return rrmse_loss

def calculate_errors(y, y_pred):
    MaxError = max_error                         (y, y_pred)
    MedError = median_absolute_error             (y, y_pred)
    MAE = mean_absolute_error                    (y, y_pred)
    MSE = mean_squared_error                     (y, y_pred)
    RMSE = mean_squared_error                    (y, y_pred, squared=False)
    RRMSE = relative_root_mean_squared_error     (y, y_pred)
    R2 = r2_score                                (y, y_pred)

    return {
            'MaxError' : round(MaxError, 5),
            'MedError' : round(MedError, 5),
            'MAE' :      round(MAE, 5),
            'MSE' :      round(MSE, 5),
            'RMSE' :     round(RMSE, 5),
            'RRMSE' :    round(RRMSE, 5),
            'R2' :       round(R2, 5)}

In [11]:
######### TODO: Refactor
# Load all data with pandas
data = pd.read_csv('/content/Tunneling_Induced_building_damage_dataset.txt', sep='\t')
data = data.drop(labels = 'Tot No. Simulations', axis=1)
# remove outliers
data = data[(np.abs(stats.zscore(data.select_dtypes(exclude='object'))) < 3).all(axis=1)]
data_columns = data.columns
data.head()
X = data.iloc[:,:15].to_numpy()
if task_name == 'Task1':
    Y = data.iloc[:,[29, 30]]
elif task_name == 'Task2':
    Y = data.iloc[:,15:]
    Y = Y.iloc[:,1:-2]
target_columns = Y.columns
Y = Y.to_numpy()
######### END: Refactor

model_1_errors_train = []
model_1_errors_test = []
model_2_errors_train = []
model_2_errors_test = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X, Y):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

    scaler_x = RobustScaler()
    scaler_y = RobustScaler()

# {'target': 1.7095960000000001, 'params': {
#     'l2_regularization_1': 0.09999969621218786,
#     'l2_regularization_2': 0.04025532063670965,
#     'learning_rate_1': 0.06343280521222003,
#     'learning_rate_2': 0.06180901311792841,
#     'max_bins_1': 163.16001259018387,
#     'max_bins_2': 198.1389770409653,
#     'max_iter_1': 240.31073937395976,
#     'max_iter_2': 761.8495697650524,
#     'number_of_steps': 12.347814136890445
# }}

    model1 = ModelStrategy(HistGradientBoostingRegressor(
        learning_rate = 0.06343280521222003,
        l2_regularization = 0.09999969621218786,
        max_iter = 240,
        max_bins = 163,
        random_state=42
        ))
    model2 = AdapterSGTM(number_of_steps=12, verbose=-1)
    model3 = ModelStrategy(HistGradientBoostingRegressor(
        learning_rate = 0.06180901311792841,
        l2_regularization = 0.04025532063670965,
        max_iter = 761,
        max_bins = 198,
        random_state=42
        ))

    X_train = scaler_x.fit_transform(X_train)
    Y_train_scaled = scaler_y.fit_transform(Y_train)
    X_test = scaler_x.transform(X_test)

    # Model 1 fit on the model
    Y_train_pred = model1.fit_transform(X_train, Y_train_scaled)
    Y_test_pred = model1.transform(X_test)

    # Use decomposition to increase input features by merge X_train & Y_pred
    X_train_new = model2.fit_transform(np.concatenate((X_train, Y_train_pred), axis=1))
    X_test_new = model2.transform(np.concatenate((X_test, Y_test_pred), axis=1))

    # Model 2 fit on the model
    Y_train_pred_new = model3.fit_transform(X_train_new, Y_train_scaled)
    Y_test_pred_new = model3.transform(X_test_new)

    for i, target_name in enumerate(target_columns):
      # Model 1 (Save results for calculating errors)
      model_1_errors_train.append({f'{target_name}' : calculate_errors(Y_train.T[i], scaler_y.inverse_transform(Y_train_pred).T[i])})
      model_1_errors_test.append({f'{target_name}' : calculate_errors(Y_test.T[i], scaler_y.inverse_transform(Y_test_pred).T[i])})

      # Model 2 (Save results for calculating errors)
      model_2_errors_train.append({f'{target_name}' : calculate_errors(Y_train.T[i], scaler_y.inverse_transform(Y_train_pred_new).T[i])})
      model_2_errors_test.append({f'{target_name}' : calculate_errors(Y_test.T[i], scaler_y.inverse_transform(Y_test_pred_new).T[i])})

In [12]:
class AveragesCalculator:
    def __init__(self, data):
        self.data = data

    def _split_data(self):
        splitted_data = {}

        for dictionary in self.data:
            for key, inner_dict in dictionary.items():
                if key not in splitted_data:
                    splitted_data[key] = []
                splitted_data[key].append(inner_dict)

        return splitted_data

    def _sum_values(self, data):
        sums = {}

        for key, values in data.items():
            sums[key] = {}
            num_elements = len(values)
            for dictionary in values:
                for inner_key, value in dictionary.items():
                    if inner_key in sums[key]:
                        sums[key][inner_key] += value
                    else:
                        sums[key][inner_key] = value
            # Store the number of elements for each key
            sums[key]['num_elements'] = num_elements

        return sums

    def _compute_averages(self, sums):
        averages = {}

        for key, value in sums.items():
            averages[key] = {}
            num_elements = value.pop('num_elements')  # Remove num_elements from the dictionary
            for inner_key, inner_value in value.items():
                averages[key][inner_key] = inner_value / num_elements

        return averages

    def calculate_averages(self):
        splitted_data = self._split_data()
        sums = self._sum_values(splitted_data)
        averages = self._compute_averages(sums)

        return averages


def add_sufix_to_keys(dictionary, sufix):
    return {key + sufix : value for key, value in dictionary.items()}

# Calculate the averages
_model_1_errors_train = AveragesCalculator(model_1_errors_train).calculate_averages()
_model_1_errors_test = AveragesCalculator(model_1_errors_test).calculate_averages()
_model_2_errors_train = AveragesCalculator(model_2_errors_train).calculate_averages()
_model_2_errors_test = AveragesCalculator(model_2_errors_test).calculate_averages()

_model_1_errors_train = add_sufix_to_keys(_model_1_errors_train, '_train_1')
_model_1_errors_test = add_sufix_to_keys(_model_1_errors_test, '_test_1')
_model_2_errors_train = add_sufix_to_keys(_model_2_errors_train, '_train_2')
_model_2_errors_test = add_sufix_to_keys(_model_2_errors_test, '_test_2')

results = {
    **dict(sorted({**_model_1_errors_train, **_model_1_errors_test}.items(), reverse=True)),
    **dict(sorted({**_model_2_errors_train, **_model_2_errors_test}.items(), reverse=True))
}

results = pd.DataFrame(results)
results.to_excel(f'{task_name}_KFold_Decomposition{additional_notes}.xlsx')
results

Local Avg._train_1  Local Avg._test_1  Global Avg._train_1  \
MaxError        1.395650e+00       5.977808e+00             0.000358   
MedError        6.613400e-02       7.130240e-01             0.000010   
MAE             1.061220e-01       1.121682e+00             0.000020   
MAPE            2.146830e+14       1.952825e+15             0.077728   
MSE             2.869000e-02       2.678610e+00             0.000000   
RMSE            1.692740e-01       1.632688e+00             0.000038   
RRMSE           1.526000e-03       3.293400e-02             0.001008   
R2              9.974520e-01       7.531380e-01             0.998252   

          Global Avg._test_1  Local Avg._train_2  Local Avg._test_2  \
MaxError            0.001298        4.768780e-01       5.584442e+00   
MedError            0.000138        1.694000e-03       2.730620e-01   
MAE                 0.000208        6.332000e-03       8.718360e-01   
MAPE                0.564060        6.081343e+12       1.008007e+15   
MSE                 0.000000        6.020000e-04       2.070558e+00   
RMSE                0.000304        2.373000e-02       1.436682e+00   
RRMSE               0.016682        2.100000e-04       2.823600e-02   
R2                  0.887578        9.999480e-01       8.074420e-01   

          Global Avg._train_2  Global Avg._test_2  
MaxError             0.000116            0.001478  
MedError             0.000000            0.000104  
MAE                  0.000000            0.000186  
MAPE                 0.005002            0.391430  
MSE                  0.000000            0.000000  
RMSE                 0.000008            0.000282  
RRMSE                0.000176            0.015636  
R2                   0.999940            0.902154